# 팔당댐 홍수 안전운영에 따른 한강 수위예측 AI 경진대회

### 필요한 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np

from glob import glob
from tqdm import tqdm
from scipy import interpolate

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, AveragePooling1D, GlobalAveragePooling1D

'''#GPU 자원이 부족한 경우 아래 코드를 이용하세요
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)'''

'#GPU 자원이 부족한 경우 아래 코드를 이용하세요\nfrom tensorflow.compat.v1 import ConfigProto\nfrom tensorflow.compat.v1 import InteractiveSession\n\nconfig = ConfigProto()\nconfig.gpu_options.allow_growth = True\nsession = InteractiveSession(config=config)'

### 데이터 전처리

In [2]:
w_list = sorted(glob("competition_data/water_data/*.csv"))
w_list

['competition_data/water_data\\data_2012.csv',
 'competition_data/water_data\\data_2013.csv',
 'competition_data/water_data\\data_2014.csv',
 'competition_data/water_data\\data_2015.csv',
 'competition_data/water_data\\data_2016.csv',
 'competition_data/water_data\\data_2017.csv',
 'competition_data/water_data\\data_2018.csv',
 'competition_data/water_data\\data_2019.csv',
 'competition_data/water_data\\data_2020.csv',
 'competition_data/water_data\\data_2021.csv',
 'competition_data/water_data\\data_2022.csv']

In [3]:
pd.read_csv(w_list[0]).shape

(26496, 15)

# 년월일시분 / 팔당댐 현재 수위 / 팔당댐 유입량 / 팔당댐 저수량 / 팔당댐 공용량 / 총 방류량 / 강화대교 조위 / 청담대교 수위 / 청담대교 유량 / 잠수교 수위 / 잠수교 유량 / 한강대교 수위 / 행주대교 수위 / 행주대교 유량

In [4]:
pd.read_csv(w_list[0]).head(4)

,ymdhm,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630
0,2012-05-01 00:00,24.800,555.0,219.07,24.93,555.0,445.0,310.7,469.05,300.2,0.0,290.0,729.80,275.3,540.18
1,2012-05-01 00:10,24.794,464.6,218.86,25.15,562.9,449.0,314.7,498.00,300.2,0.0,290.0,731.48,275.3,540.18
2,2012-05-01 00:20,24.789,478.1,218.69,25.31,576.4,451.0,313.7,490.68,301.2,0.0,290.0,726.42,275.3,540.18
3,2012-05-01 00:30,24.789,464.8,218.69,25.31,563.1,452.0,311.7,476.21,301.2,0.0,290.0,726.42,276.3,552.17


In [5]:
train_data = []
train_label = []
num = 0

for i in w_list[:-1]:
    
    tmp = pd.read_csv(i)
    tmp = tmp.replace(" ", np.nan)
    tmp = tmp.interpolate(method = 'values')
    tmp = tmp.fillna(0)
    
    for j in tqdm(range(len(tmp)-432)):
        train_data.append(np.array(tmp.loc[j:j + 431, ["swl", "inf", "sfw", "ecpc",
                                                       "tototf", "tide_level",
                                                       "fw_1018662", "fw_1018680",
                                                       "fw_1018683", "fw_1019630"]]).astype(float))
        
        train_label.append(np.array(tmp.loc[j + 432, ["wl_1018662", "wl_1018680",
                                                      "wl_1018683", "wl_1019630"]]).astype(float))

100%|██████████████████████████████████████████████████████████████████████████| 26064/26064 [00:15<00:00, 1707.38it/s]


In [ ]:
from sklearn.preprocessing import MinMaxScaler
transformer = MinMaxScaler()
transformer.fit(train_data)
train_data = transformer.transform(train_data)

In [6]:
train_data = np.array(train_data)
train_label = np.array(train_label)

print(train_data.shape)
print(train_label.shape)

(260640, 432, 10)
(260640, 4)


### 모델링 및 모델 학습

In [7]:
input_shape = (train_data[0].shape[0], train_data[0].shape[1])

model = Sequential()
model.add(GRU(256, input_shape=input_shape))
model.add(Dense(4, activation = 'relu'))

optimizer = tf.optimizers.RMSprop(0.001)

model.compile(optimizer=optimizer,loss='mse', metrics=['mae'])


In [8]:
model.fit(train_data, train_label, epochs=10, batch_size=128)

Epoch 1/10
2037/2037 [==============================] - 71s 31ms/step - loss: 22922.6699 - mae: 103.4684
Epoch 2/10
2037/2037 [==============================] - 63s 31ms/step - loss: 656.0679 - mae: 9.1563
Epoch 3/10
2037/2037 [==============================] - 63s 31ms/step - loss: 432.7420 - mae: 7.9663
Epoch 4/10
2037/2037 [==============================] - 64s 31ms/step - loss: 362.6096 - mae: 7.5787
Epoch 5/10
2037/2037 [==============================] - 63s 31ms/step - loss: 305.6074 - mae: 7.5019
Epoch 6/10
2037/2037 [==============================] - 63s 31ms/step - loss: 293.7475 - mae: 7.5123
Epoch 7/10
2037/2037 [==============================] - 63s 31ms/step - loss: 254.6130 - mae: 7.5706
Epoch 8/10
2037/2037 [==============================] - 63s 31ms/step - loss: 253.5053 - mae: 7.7600
Epoch 9/10
2037/2037 [==============================] - 63s 31ms/step - loss: 245.7018 - mae: 7.8559
Epoch 10/10
2037/2037 [==============================] - 63s 31ms/step - loss: 235.3584

### 추론 데이터셋 만들기

In [9]:
test_data = []
test_label = []

tmp = pd.read_csv(w_list[-1])
tmp = tmp.replace(" ", np.nan)
# 이전값을 사용
tmp = tmp.fillna(method = 'pad')
tmp = tmp.fillna(0)
    
#tmp.loc[:, ["wl_1018662", "wl_1018680", "wl_1018683", "wl_1019630"]] = tmp.loc[:, ["wl_1018662", "wl_1018680", "wl_1018683", "wl_1019630"]]*100
    
for j in tqdm(range(4032, len(tmp)-432)):
    test_data.append(np.array(tmp.loc[j:j + 431, ["swl", "inf", "sfw", "ecpc",
                                                    "tototf", "tide_level",
                                                    "fw_1018662", "fw_1018680",
                                                    "fw_1018683", "fw_1019630"]]).astype(float))
        
    test_label.append(np.array(tmp.loc[j + 432, ["wl_1018662", "wl_1018680",
                                                    "wl_1018683", "wl_1019630"]]).astype(float))

100%|████████████████████████████████████████████████████████████████████████████| 6912/6912 [00:04<00:00, 1607.83it/s]


In [10]:
test_data = np.array(test_data)
test_label = np.array(test_label)

print(test_data.shape)
print(test_label.shape)

(6912, 432, 10)
(6912, 4)


### 제출 파일 만들기

In [11]:
pred = model.predict(test_data)

In [12]:
pred = pd.DataFrame(pred)

In [13]:
sample_submission = pd.read_csv("competition_data/sample_submission.csv")

sample_submission["wl_1018662"] = pred[0]
sample_submission["wl_1018680"] = pred[1]
sample_submission["wl_1018683"] = pred[2]
sample_submission["wl_1019630"] = pred[3]

In [14]:
sample_submission.to_csv("baseline.csv", index = False)